<a href="https://colab.research.google.com/github/vishaljoshi24/DnD-AutoPrompt/blob/main/behaviour_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 33.6 MB/s eta 0:00:00


In [2]:
import dspy

lm = dspy.LM("ollama_chat/llama3.2:1b", api_base = "http://localhost:11434", api_key="b229890ea0664f6193770b4b470f3e74.nJRqYf9TBF9MX24r4fUQQqqB")
dspy.configure(lm=lm)

In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("garrykuwanto/crd3_training_pairs", download_mode="force_redownload")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


crd3_training_pairs_no_llm.json:   0%|          | 0.00/773M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2360582 [00:00<?, ? examples/s]

In [4]:
df = pd.DataFrame(dataset)
query = []
passage = []
for i in range(len(df.train[0:])):
  query.append(df.train[i]['query'])
  passage.append(df.train[i]['passage'])

df1 = pd.DataFrame({'context': query, 'current turn': passage})

In [7]:
examples = []

for context, turn in df1.values:
  examples.append(dspy.Example(context=context, turn=turn).with_inputs("context", "current turn"))

In [8]:
import random

random.Random(0).shuffle(examples)

In [9]:
examples[:10]

[Example({'context': "Scene context: Taliesin tells them that the coolest part is the back. Travis flips it over to reveal Laura, advertising the Slayer's Cake apron.", 'turn': "LAURA: Hey, and it's the Slayer's Cake apron that I was just--"}) (input_keys={'context', 'current turn'}),
 Example({'context': 'Player says: Roll high. Come on.', 'turn': "DM responds: I say with advantage because Trinket's been your companion for so long."}) (input_keys={'context', 'current turn'}),
 Example({'context': "Dialogue context: TRAVIS: Yeah, you knock down the barriers, you get right in. Icebreaker. SAM: People hate me. TRAVIS: No, that's not true at all.", 'turn': 'LIAM: I will go with you, if you want. If you want us to ask this--'}) (input_keys={'context', 'current turn'}),
 Example({'context': "Dialogue context: MARISHA: Look again! Abigail Persimmon. TALIESIN: The magic of the goth makeover strikes again! (cackles) MARISHA: I'm not coming tomorrow because I've already had mine.", 'turn': 'TAL

In [11]:
from typing import Literal

class behaviour_classifier(dspy.Signature):
    examples: dict[str, str] = dspy.InputField(desc = """
  {'context': 'The three previous game turns which describe a player's action or their dialogue.',
   'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
   }""")
    behaviour_class: Literal['collaborative action', 'contextually-relevant action', 'goal-oriented action', 'open-ended action'] = dspy.OutputField(desc="The feature of a player's actions within a Dungeons & Dragons game: collaborative, open-ended, goal-oriented, contextually relevant")

classify = dspy.ChainOfThought(behaviour_classifier)


In [13]:
from dspy.clients.lm_local import LocalProvider
student_lm = lm
teacher_lm = dspy.LM('openai/gpt-4o-mini', max_tokens=3000)

In [14]:
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

teacher_classify = classify.deepcopy()
teacher_classify.set_lm(teacher_lm)